# 残差网络（ResNet）

让我们先思考一个问题：对神经网络模型添加新的层，充分训练后的模型是否只可能更有效地降低训练误差？理论上，原模型解的空间只是新模型解的空间的子空间。也就是说，如果我们能将新添加的层训练成恒等映射$f(x) = x$，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。针对这一问题，何恺明等人提出了残差网络（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。


## 残差块

让我们聚焦于神经网络局部。如图5.9所示，设输入为$\boldsymbol{x}$。假设我们希望学出的理想映射为$f(\boldsymbol{x})$，从而作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$。残差映射在实际中往往更容易优化。以本节开头提到的恒等映射作为我们希望学出的理想映射$f(\boldsymbol{x})$。我们只需将图5.9中右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0，那么$f(\boldsymbol{x})$即为恒等映射。实际中，当理想映射$f(\boldsymbol{x})$极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。图5.9右图也是ResNet的基础块，即残差块（residual block）。在残差块中，输入可通过跨层的数据线路更快地向前传播。

![设输入为$\boldsymbol{x}$。假设图中最上方激活函数输入的理想映射为$f(\boldsymbol{x})$。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$](../img/residual-block.svg)

ResNet沿用了VGG全$3\times 3$卷积层的设计。残差块里首先有2个有相同输出通道数的$3\times 3$卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的$1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。

残差块的实现如下。它可以设定输出通道数、是否使用额外的$1\times 1$卷积层来修改通道数以及卷积层的步幅。

In [0]:
import d2ltorch as d2lt
import torch
from torch import nn, optim
from torchsummary import summary

class Residual(nn.Module): # 本类已保存在d2ltorch包中方便以后使用
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, 
                               stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                  stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
    def forward(self, X):
        Y = torch.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return torch.relu(Y + X)

下面我们来查看输入和输出形状一致的情况。

In [74]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
blk(X).shape

torch.Size([4, 3, 6, 6])

我们也可以在增加输出通道数的同时减半输出的高和宽。

In [75]:
blk = Residual(3, 6, use_1x1conv=True, stride=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

## ResNet模型

ResNet与之前介绍的GoogleLeNet的结构类似，与GoogLeNet中4个由Inception块组成的模块对应，ResNet使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。

In [0]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0:
            if not first_block:
                blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
            else:
                blk.append(Residual(in_channels, out_channels))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

下面仿照GoogLeNet实现ResNet。

In [0]:
class MyResNet(nn.Module):
    def __init__(self, in_channels, **kwargs):
        super(MyResNet, self).__init__(**kwargs)
        
        # ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、
        # 步幅为2的 7 x 7卷积层后接步幅为2的 3 x 3 的最大池化层。
        # 不同之处在于ResNet每个卷积层后增加的批量归一化层。
        self.b1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。
        # 然后加一个全局平均池化层调整大小
        self.b2 = nn.Sequential(
            resnet_block(64, 64, 2, first_block=True),
            resnet_block(64, 128, 2),
            resnet_block(128, 256, 2),
            resnet_block(256, 512, 2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        # 最后，与GoogLeNet一样，接上全连接层输出。
        self.output = nn.Linear(512*1*1, 10)
    
    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = x.reshape(-1, 512*1*1)
        x = self.output(x)
        return x

这里每个模块里有4个卷积层（不计算$1\times 1$卷积层），加上最开始的卷积层和最后的全连接层，共计18层。这个模型通常也被称为ResNet-18。通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的ResNet-152。虽然ResNet的主体架构跟GoogLeNet的类似，但ResNet结构更简单，修改也更方便。这些因素都导致了ResNet迅速被广泛使用。

在训练ResNet之前，我们来观察一下输入形状在ResNet不同模块之间的变化。

In [78]:
net = MyResNet(in_channels=1)
summary(net, (1, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,200
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,928
       BatchNorm2d-6           [-1, 64, 56, 56]             128
            Conv2d-7           [-1, 64, 56, 56]          36,928
       BatchNorm2d-8           [-1, 64, 56, 56]             128
          Residual-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,928
      BatchNorm2d-11           [-1, 64, 56, 56]             128
           Conv2d-12           [-1, 64, 56, 56]          36,928
      BatchNorm2d-13           [-1, 64, 56, 56]             128
         Residual-14           [-1, 64,

## 获取数据和训练模型

下面我们在Fashion-MNIST数据集上训练ResNet。

In [79]:
root = '~/dataset/'
lr, num_epochs, batch_size, device = 0.05, 5, 256, d2lt.try_gpu()

d2lt.params_init(net, init=nn.init.xavier_uniform_)

optimizer = optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2lt.load_data_fashion_mnist(root, batch_size, resize=96)
d2lt.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device,
              num_epochs)

training on gpu
epoch 1, loss 0.0019, train acc 0.830, test acc 0.871, time 47.2 sec
epoch 2, loss 0.0010, train acc 0.906, test acc 0.883, time 49.0 sec
epoch 3, loss 0.0007, train acc 0.930, test acc 0.858, time 50.7 sec
epoch 4, loss 0.0006, train acc 0.947, test acc 0.869, time 51.5 sec
epoch 5, loss 0.0004, train acc 0.961, test acc 0.894, time 51.4 sec


## 小结

* 残差块通过跨层的数据通道从而能够训练出有效的深度神经网络。
* ResNet深刻影响了后来的深度神经网络的设计。


## 练习

* 参考ResNet论文的表1来实现不同版本的ResNet [1]。
* 对于比较深的网络， ResNet论文中介绍了一个“瓶颈”架构来降低模型复杂度。尝试实现它 [1]。
* 在ResNet的后续版本里，作者将残差块里的“卷积、批量归一化和激活”结构改成了“批量归一化、激活和卷积”，实现这个改进（[2]，图1）。



## 参考文献

[1] He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 770-778).

[2] He, K., Zhang, X., Ren, S., & Sun, J. (2016, October). Identity mappings in deep residual networks. In European Conference on Computer Vision (pp. 630-645). Springer, Cham.

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1663)

![](../img/qr_resnet.svg)